#**Keras entradas múltimples y datos mixtos - Tutorial 2**

Tutorial 2 de PyimageSearch de cómo hacer una predicción del precio de la vivienda en Keras usando datos e imágenes.

La primera parte es un perceptrón multicapa MLP manejando valores categóricos y numéricos. La segunda parte es una red neuronal convolucional CNN para trabajar datos de imagen. Ambas se concatenarán.

In [1]:
# Primero cargamos librerías y funciones necesarias
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os

#Para trabajar en el colab
from google.colab import drive
drive.mount('/content/drive')

def load_house_attributes():
    # Nombre de las columnas del DF
    cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
    # Carga del DF desde el fichero de texto HousesInfo.txt, poniéndole a las columnas el nombre definido antes
    df = pd.read_csv('/content/drive/My Drive/BootCamp - BigDataIV - DL/HousesInfo.txt', sep=' ', decimal='.', header=None, names=cols)

    # Si hay menos de 25 casas por cada código postal, se elimina.
    zipcodes = df["zipcode"].value_counts().keys().tolist() # Lista de los CP
    counts = df["zipcode"].value_counts().tolist() # Número de veces que aparece cada CP anterior

    # Recorremos cada CP y el número de veces que aparece. Si es menos de 25, se elimina los reg con ese CP
    for (zipcode, count) in zip(zipcodes, counts):
        if count < 25:
            idxs = df[df["zipcode"] == zipcode].index
            df.drop(idxs, inplace=True)

    # Devuelve el DF
    return df


def process_house_attributes(df, train, test):
    # Nombre de las columnas del DF variables continuas
    continuous = ["bedrooms", "bathrooms", "area"]

    # Escalar cada columna continua en el rango [0,1], en train y test
    cs = MinMaxScaler()
    trainContinuous = cs.fit_transform(train[continuous])
    testContinuous = cs.transform(test[continuous])

    # Idem para la columna código postal, usando one-hot, y transformarlo a un array
    zipBinarizer = LabelBinarizer().fit(df["zipcode"])
    trainCategorical = zipBinarizer.transform(train["zipcode"])
    testCategorical = zipBinarizer.transform(test["zipcode"])

    # Unimos categóricas y continuas en un solo array
    trainX = np.hstack([trainCategorical, trainContinuous])
    testX = np.hstack([testCategorical, testContinuous])

    # devuelve train y test concatenados
    return (trainX, testX)


def load_house_images(df):
    # Procedemos a la carga de imágenes desde el colab. Cuatro fotos por casa
    images = []

    # recorremos los índices de los 362 registros del DF (recordemos que hubo una purga del CP) 
    for i in df.index.values:
        # las imágenes son 4 de cada registro, se recuperan en el mismo orden desde el colab
        basePath = os.path.sep.join(['/content/drive/My Drive/BootCamp - BigDataIV - DL/Houses Dataset/', "{}_*".format(i + 1)])
        housePaths = sorted(list(glob.glob(basePath)))

        inputImages = []
        outputImage = np.zeros((64, 64, 3), dtype="uint8")

        # Recorre todas las fotos recuperadas arriba
        for housePath in housePaths:
            image = cv2.imread(housePath)        # lee la imagen
            image = cv2.resize(image, (32, 32))  # ajusta el tamaño a 32x32
            inputImages.append(image)            # añade la imagen al array creado antes

        outputImage[0:32, 0:32] = inputImages[0]    #arriba dcha
        outputImage[0:32, 32:64] = inputImages[1]   #arriba izda 
        outputImage[32:64, 32:64] = inputImages[2]  #abajo dcha
        outputImage[32:64, 0:32] = inputImages[3]   #abajo izda

        # la foto creada se añade al array de salida
        images.append(outputImage)

    # devuelve set de imágenes
    return np.array(images)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model

# Creamos un perceptrón multicapa (MLP), compilar el modelo usando el mean absolute percentage error para
# las pérdidas, buscando minimizar la diferencia entre nuestras predicciones de precios y los precios actuales
def create_mlp(dim, regress=False):
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu")) #La dimensión de entrada será la del número de columnas de train
    model.add(Dense(4, activation="relu"))   # 4 neuronas con función de activación relu
    model.add(Dense(1, activation="linear")) # neurona con función de activación lineal

    # devuelve el modelo
    return model

# CNN para datos de imagen
def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
    inputShape = (height, width, depth) # tamaño datos entrada : fotos 64x64 y 3 de foto RGB
    chanDim = -1                        # Canal dimensión

    inputs = Input(shape=inputShape) # Entrada para el modelo

    # Recorre todos los filtros
    for (i, f) in enumerate(filters):
        if i == 0:           # Si es la primera capa
            x = inputs

        # CONV => RELU => BN => POOL se añaden las capas siguientes para cada filtro
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

    # FC => RELU => BN => DROPOUT sigue añadiendo capas
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)

    # capas para para que coincida con el MLP
    x = Dense(4)(x)
    x = Activation("relu")(x)

    # construye la red CNN
    model = Model(inputs, x)

    # devuelve la CNN
    return model


Using TensorFlow backend.


Se combinan MLP y CNN anteriores

In [3]:

# import the necessary packages
#from pyimagesearch import datasets
#from pyimagesearch import models
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
import numpy as np
import argparse
import locale
import os

b = 'b'

print("[INFO] loading house attributes...")
# Carga en un DF los datos de house.txt para la MLP
df = load_house_attributes()


# Carga las imágenes de las casas
print("[INFO] loading house images...")
images = load_house_images(df)
images = images / 255.0

# se dividen los datos en train y test, tanto de la info de las casas como de las fotos 
print("[INFO] processing data...")
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

# Se categoriza el precio de train y test
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

# Categorizamos las variables de las casas
(trainAttrX, testAttrX) = process_house_attributes(df, trainAttrX, testAttrX)

# crea los modelos MLP y CNN llamando a las funciones de arriba
mlp = create_mlp(trainAttrX.shape[1], regress=False)
cnn = create_cnn(64, 64, 3, regress=False)

# combina ambas salidas de las funciones
combinedInput = concatenate([mlp.output, cnn.output])

# añade dos nuevas capas a la combinada
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

# el mododelo final 
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

# compila el modelo construido, usando mean abs per error para las pérdidas,
# que busca minimizar el porcentaje de la diferencia absoluta entre el precio 
# predecido y el actual
opt = Adam(lr=1e-3, decay=1e-3 / 200) # optimizador algoritmo Adam, learning rate bajo, disminución exp
model.compile(loss="mean_absolute_percentage_error", optimizer=opt) # compila pérdidas con modelo mabspererror

# entrenamiento del modelo
print("[INFO] training model...")
model.fit(
    [trainAttrX, trainImagesX], trainY,
    validation_data=([testAttrX, testImagesX], testY),
    epochs=200, batch_size=8)

# predicciones sobre train test
print("[INFO] predicting house prices...")
preds = model.predict([testAttrX, testImagesX])

# Calcula la diferencia entre los precios predecidos y los actuales 
diff = preds.flatten() - testY        # diferencia
percentDiff = (diff / testY) * 100    # porcentaje
absPercentDiff = np.abs(percentDiff)  # valor absoluto del porcentaje de la diferencia

# Media y desviación estandard del anterior
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# Estadísticas del modelo
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
    locale.currency(df["price"].mean(), grouping=True),
    locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] loading house attributes...
[INFO] loading house images...
[INFO] processing data...












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

[INFO] training model...


Train on 271 samples, validate on 91 samples
Epoch 1/200
271/271 [==============================] - 3s 13ms/step - loss: 371.3046 - val_loss: 198.7709
Epoch 2/200
271/271 [==============================] - 2s 8ms/step - loss: 184.3103 - val_loss: 82.0429
Epoch 3/200
271/271 [==============================] - 2s 8ms/step - loss: 101.9289 - val_loss: 73.8325
Epoch 4/200
271/271 [==============================] - 2s 8ms/step - loss: 93.5560 - val_loss: 71.5834
Epoch 5/200
271/271 [==============================] - 2s 8ms/step - loss: 82.1281 - val_loss: 70.1846
Epoch 6/200
271/271 [==============================] - 2s 8ms/step - loss: 74.2788 - val_loss: 69.2097
Epoch 7/200
271/271 [==============================] - 2s 8ms/step - loss: 74.5952 - 

Nuestro error porcentual absoluto empieza altísimo, pero cae durante el proceso. Al final de la capacitación, estamos obteniendo un 21.15% de error porcentual absoluto medio. Un resultado bastante parecido respecto a la regresión. Este modelo apenas ha mejorado, tanto en este colab, como en los tutoriales pasados (en esos tutoriales, pasar de regresión básica de 26% a 23 % a pesar de meterles las imágenes es un triunfo).


Así que siendo directos, dado que tanto yo como los tutoriales hemos obtenidos resultados parecidos, concluyo que en modelo mixto de MLP + CNN realizado no está bien aprovechado, y habría que probar más parámetros, modelos e índices.

